# Original code from the class
- Divides the data into a training set (last quarter of 2018) and a test set (first day of 2019) based on date ranges.

In [ ]:
# import pandas as pd
# import numpy as np

# df = pd.read_csv("Bitstamp_BTCUSD_2018_minute.csv", header=1)
# df.index = df.date
# df.drop(["unix", "date", "symbol", "Volume USD"], axis=1, inplace=True)
# df = df.iloc[::-1] # Reverse the order of dates

# BTC_vol = df["Volume BTC"].values
# df_diff = df.diff()
# df_diff["Volume BTC"] = np.log(1 + BTC_vol)

# # Select the last quarter as the training dataset
# date_index = pd.to_datetime(df_diff.index)
# mask_train = pd.Series(date_index).between("2018-10-01", "2018-12-31",
#     inclusive="left")
# df_train = df_diff.loc[mask_train.values].copy(deep=True)
# y_close_train = df.close.loc[mask_train.values]
# train_close = df.loc[mask_train.values].close.values
# df_train["Relative_Close"] = train_close / train_close[0]

# # Select the first day as the test dataset
# mask_test = pd.Series(date_index).between("2018-12-31", "2019-01-01",
#     inclusive="left")
# df_test = df_diff.loc[mask_test.values].copy(deep=True)
# test_close = df.loc[mask_test.values].close.values
# df_test["Relative_Close"] = test_close / train_close[0]
# y_close_test = df.close[mask_test.values]

# print(df_train.columns.values)

# def generate_dataset(df, seq_len):
#     X_list, y_list = [], []
#     for i in range(len(df.index) - seq_len):
#         X_list.append(np.array(df.iloc[i:i+seq_len,:]))
#         y_list.append(df.close.values[i+seq_len])
#     return np.array(X_list), np.array(y_list)

In [ ]:
# LAG = 5
# # Add LAG number of observations in training dataset to test dataset
# df_test_LAG = pd.concat((df_train.iloc[-LAG:,:], df_test), axis=0)

# X_train, y_train = generate_dataset(df_train, seq_len=LAG)
# X_test, y_test = generate_dataset(df_test_LAG, seq_len=LAG)

# print(np.mean(y_train))
# print(np.mean((y_train - np.mean(y_train))**2))
# print(np.mean((y_test - np.mean(y_train))**2))

# import tensorflow as tf
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.optimizers import Adam

# tf.keras.utils.set_random_seed(4002)

# model = Sequential()
# model.add(LSTM(8, return_sequences=True))
# model.add(LSTM(8, dropout=0.2))

# model.add(Dense(1))
# model.@@compile@@(optimizer=Adam(learning_rate=0.001), loss='mse')
# model.fit(X_train, y_train, batch_size=64, epochs=30, shuffle=True)

# LSTM_pred = np.squeeze(model.predict(X_test))
# print(np.mean((LSTM_pred - y_test)**2))

## Changed to daily basis

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
df = pd.read_csv('/content/Bitcoin Historical Data3 2 (1).csv')

In [ ]:
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,03/10/2024,"69,437.1","68,360.7","69,676.4","68,250.3",37.43K,1.57%
1,03/09/2024,"68,366.5","68,178.5","68,576.9","67,923.9",30.71K,0.29%
2,03/08/2024,"68,172.0","66,854.4","69,904.0","66,170.7",112.67K,1.97%
3,03/07/2024,"66,855.3","66,074.6","67,985.5","65,602.6",77.47K,1.17%
4,03/06/2024,"66,080.4","63,794.7","67,604.9","62,848.7",117.91K,3.59%


In [ ]:
df.columns = ['date', 'close', 'open', 'high', 'low', 'vol', 'change']

### Data preprocessing

In [ ]:
def percentage_to_decimal(s):
    return float(s.strip('%')) / 100

In [ ]:
def convert_volume(volume):

    volume_str = str(volume)

    if 'K' in volume_str:
        return float(volume_str.replace('K', '')) * 1e3
    elif 'M' in volume_str:
        return float(volume_str.replace('M', '')) * 1e6
    elif 'B' in volume_str:
        return float(volume_str.replace('B', '')) * 1e9
    else:
        return float(volume_str)

In [ ]:
df['vol'] = df['vol'].apply(convert_volume)
df['close'] = df['close'].replace(',', '', regex=True).astype(float)
df['open'] = df['open'].replace(',', '', regex=True).astype(float)
df['high'] = df['high'].replace(',', '', regex=True).astype(float)
df['low'] = df['low'].replace(',', '', regex=True).astype(float)
df['change'] = df['change'].apply(percentage_to_decimal)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4985 entries, 0 to 4984
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    4985 non-null   object 
 1   close   4985 non-null   float64
 2   open    4985 non-null   float64
 3   high    4985 non-null   float64
 4   low     4985 non-null   float64
 5   vol     4979 non-null   float64
 6   change  4985 non-null   float64
dtypes: float64(6), object(1)
memory usage: 272.7+ KB


In [ ]:
df.index = pd.to_datetime(df['date'])

In [ ]:
df.head()

,date,close,open,high,low,vol,change
date,,,,,,,
2024-03-10,03/10/2024,69437.1,68360.7,69676.4,68250.3,37430.0,0.0157
2024-03-09,03/09/2024,68366.5,68178.5,68576.9,67923.9,30710.0,0.0029
2024-03-08,03/08/2024,68172.0,66854.4,69904.0,66170.7,112670.0,0.0197
2024-03-07,03/07/2024,66855.3,66074.6,67985.5,65602.6,77470.0,0.0117
2024-03-06,03/06/2024,66080.4,63794.7,67604.9,62848.7,117910.0,0.0359


In [ ]:
df.drop(["date"], axis=1, inplace=True)

In [ ]:
# Reverse the dataset order so it starts with the earliest date
df = df.iloc[::-1]

In [ ]:
# We will use data above 2017, because there are some null values in the past
df = df[df.index.year > 2017]

In [ ]:
# Prepare the volume and price differences, normalize volume
BTC_vol = df["vol"].values
df_diff = df.diff().dropna()
df_diff["vol"] = np.log(1 + BTC_vol[:-1])  # Shifted by 1 to align with diff

In [ ]:
# Assuming df_diff has been created by df.diff().dropna() or similar
# First, align the index of df to match df_diff after dropping NaN values
df_aligned = df.loc[df_diff.index]

# Now, proceed with mask creation and selection
mask_train = df_diff.index < "2018-12-01"
df_train = df_diff.loc[mask_train].copy()

# Use df_aligned to ensure the indexes match
train_close = df_aligned.loc[mask_train, "close"].values

# Since indexes are aligned, this operation should now work without error
df_train["Relative_Close"] = train_close / train_close[0]

In [ ]:
mask_test = (df_diff.index >= "2018-12-01") & (df_diff.index < "2019-01-01")  # December 2018 for testing
df_test = df_diff.loc[mask_test].copy()
test_close = df_aligned.loc[mask_test, "close"].values
df_test["Relative_Close"] = test_close / train_close[0]

In [ ]:
# Generate dataset function
def generate_dataset(df, seq_len):
    X_list, y_list = [], []
    for i in range(len(df) - seq_len):
        X_list.append(df.iloc[i:(i+seq_len), :].values)
        y_list.append(df["close"].iloc[i + seq_len])
    return np.array(X_list), np.array(y_list)

In [ ]:
LAG = 3 # Example: Use the past # days to predict the next day

In [ ]:
# Prepare training and test datasets
X_train, y_train = generate_dataset(df_train, LAG)
X_test, y_test = generate_dataset(pd.concat((df_train.iloc[-LAG:], df_test)), LAG)

In [ ]:
X_train

array([[[ 1.30920000e+03, -4.05600000e+02,  1.38460000e+03, ...,
          1.12699745e+01,  1.26700000e-01,  1.00000000e+00],
        [ 4.02500000e+02,  1.30920000e+03,  1.28900000e+02, ...,
          1.18330578e+01, -7.01000000e-02,  1.02728055e+00],
        [ 2.35000000e+01,  4.02400000e+02, -2.63000000e+01, ...,
          1.15762852e+01, -2.58000000e-02,  1.02887333e+00]],

       [[ 4.02500000e+02,  1.30920000e+03,  1.28900000e+02, ...,
          1.18330578e+01, -7.01000000e-02,  1.02728055e+00],
        [ 2.35000000e+01,  4.02400000e+02, -2.63000000e+01, ...,
          1.15762852e+01, -2.58000000e-02,  1.02887333e+00],
        [ 1.77470000e+03,  2.36000000e+01,  1.71820000e+03, ...,
          1.16170242e+01,  1.15400000e-01,  1.14915854e+00]],

       [[ 2.35000000e+01,  4.02400000e+02, -2.63000000e+01, ...,
          1.15762852e+01, -2.58000000e-02,  1.02887333e+00],
        [ 1.77470000e+03,  2.36000000e+01,  1.71820000e+03, ...,
          1.16170242e+01,  1.15400000e-01,  1.149

In [ ]:
np.mean(y_train)

-33.75878787878788

In [ ]:
print(np.mean((y_train - np.mean(y_train))**2))

152397.78175610647


In [ ]:
# Model architecture (unchanged from the minute-by-minute example)
tf.keras.utils.set_random_seed(4002)
model = Sequential([
    LSTM(8, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(8, dropout=0.2),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [ ]:
# Train the model
model.fit(X_train, y_train, batch_size=64, epochs=30, shuffle=True)

Epoch 1/30
6/6 [==============================] - 4s 7ms/step - loss: 153532.2812
Epoch 2/30
6/6 [==============================] - 0s 7ms/step - loss: 153528.8906
Epoch 3/30
6/6 [==============================] - 0s 6ms/step - loss: 153525.7188
Epoch 4/30
6/6 [==============================] - 0s 6ms/step - loss: 153522.0312
Epoch 5/30
6/6 [==============================] - 0s 6ms/step - loss: 153516.8750
Epoch 6/30
6/6 [==============================] - 0s 7ms/step - loss: 153518.9688
Epoch 7/30
6/6 [==============================] - 0s 7ms/step - loss: 153516.3750
Epoch 8/30
6/6 [==============================] - 0s 7ms/step - loss: 153517.0000
Epoch 9/30
6/6 [==============================] - 0s 6ms/step - loss: 153512.4688
Epoch 10/30
6/6 [==============================] - 0s 8ms/step - loss: 153513.0312
Epoch 11/30
6/6 [==============================] - 0s 6ms/step - loss: 153513.5312
Epoch 12/30
6/6 [==============================] - 0s 6ms/step - loss: 153513.4531
Epoch 13/30
6

In [ ]:
# Predict and evaluate
LSTM_pred = np.squeeze(model.predict(X_test))
print("Test MSE:", np.mean((LSTM_pred - y_test)**2))

1/1 [==============================] - 2s 2s/step
Test MSE: 26747.62366026146
